In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sbn
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from joblib import dump

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Прогнозирование сердечно-сосудистых заболеваний

На основе предоставленных данных спрогнозировать риск сердечно-сосудистых заболеваний (ССЗ):
* age - возраст (дней с рождения)
* gender - пол
* height - рост
* weight - вес
* ap_hi - верхнее давление
* ap_lo - нижнее давление
* cholesterol - холестерин
* gluc - глюкоза
* smoke - курит или нет
* alco - злоупотребляет алкоголем
* active - ведёт активных образ жизни
* cardio - признак наличия сердечных заболеваний

Метрика для оценки качества **ROC-AUC** по целевому признаку **cardio**

In [ ]:
df_train = pd.read_csv('train.csv',index_col=0)
df_train['is_train'] = 1
df_train.head()

In [ ]:
print(df_train.duplicated().sum())
df_train = df_train.drop_duplicates()

In [ ]:
df_test = pd.read_csv('test.csv',index_col=0)
df_test['is_train'] = 0
df_test.head()

In [ ]:
# объеденим два файла и "почистим"
df = pd.concat([df_train, df_test])

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
# для себя переименую переменные, чтобы легче было отличать бинарные от других типов
df = df.rename(columns={'gender': 'is_male', 'alco': 'is_alco', 'smoke': 'is_smoke', 'active': 'is_active', 'cardio': 'is_cardio'})

In [ ]:
df['is_male'] = df['is_male'].map({1: 0, 2: 1})

In [ ]:
df['age'] = df['age'] / 365.25
df['age'] = df['age'].astype(int)

## Анализ данных

In [ ]:
df.describe()

Выявлены следующие замечания:
* в значения роста и веса есть аномально большие и маленькие значения;
* в показателях артериального давления, есть отрицательные значения и аномально большие значения.

### Аномалии и выбросы

Нормализуем данные у данных полей:

* height
* weight
* ap_hi
* ap_lo

#### height и weight

Для анализа роста и веса добавим ИМТ (индекс массы тела)

In [ ]:
df.boxplot('height')
plt.title('Разброс роста')
plt.show()

In [ ]:
df['height'].hist(bins=100)
plt.title('Распределение роста')
plt.show()

In [ ]:
# удалим аномально высокий рост с весом 86
big_height = df[df['height'] > 240]
display(big_height.head())
df.drop(big_height.index, inplace=True)
del big_height

In [ ]:
# проверим тех у кого рост ниже 100
little_height = df[df['height'] < 100]
display(little_height.head(10))
print(little_height.shape)

Найдены слишком "маленькие люди", но с большим весом, возможно перепутали рост и вес. Исправим это.

In [ ]:
def height_weight(row):
    """
    Нормализуем значения в полях рост и вес
    
    Параметры:
    ----------
    row: Series
    
    Результат:
    ----------
    Обновлённая строка
    """
    
    if row['height'] < 100 and row['weight'] > 100:
        height = row['weight']
        weight = row['height']
        
        row['height'] = height
        row['weight'] = weight
        
    return row

df = df.apply(height_weight, axis=1)

In [ ]:
# проверяем замену
df.loc[little_height.index].head()

Аналогично проверим вес

In [ ]:
df.boxplot('weight')
plt.title('Разброс веса')
plt.show()

In [ ]:
df['weight'].hist(bins=100)
plt.title('Распределение веса')
plt.show()

In [ ]:
# проверим аномально полных и худых
display(df[df['weight'] > 180])

Показатели вполне допустимы

In [ ]:
little_weight = df[df['weight'] <= 30]
display(little_weight.head())
# удалить выбросы не получиться, но по пробуем привести их к среднему значению
df.loc[little_weight.index, 'weight'] = None

Заполним созданные пропуски

In [ ]:
# Заполним пропуск медианным начением
df['weight'] = df['weight'].fillna(df.loc[df['is_train'] == 1, 'weight'].median())

display(df.loc[little_weight.index].head())

little_weight = None

In [ ]:
df.isna().sum()

#### AP_HI и AP_LO - давление

Есть аномально большие значения. Смертельным считается 260 на 100 и 60 на 40 будем исходить из этого

In [ ]:
# начнём с отрицательных значений
df['ap_hi'] = df['ap_hi'].abs()
df['ap_lo'] = df['ap_lo'].abs()

In [ ]:
df.boxplot('ap_hi')
plt.title('Верхнее значение давления')
plt.show()

In [ ]:
df['ap_hi'].hist(bins=100)
plt.title('Верхнее значение давления')
plt.show()

In [ ]:
# разбираемся с аномально высокими значениями
big_app_hi = df[df['ap_hi'] > 800]
display(big_app_hi.sort_values('ap_hi').head())
print(big_app_hi.shape)

In [ ]:
def ap_hi(row):
    """
    Нормализуем значения в давлении
    
    Параметры:
    ----------
    row: Series
    
    Результат:
    ----------
    Обновлённая строка
    """
    
    if row['ap_hi'] > 250 and row['ap_hi'] <= 2000:
        ap_hi = int(row['ap_hi'] / 10)
        row['ap_hi'] = ap_hi
    elif row['ap_hi'] > 2000:
        ap_hi = int(row['ap_hi'] / 100)
        row['ap_hi'] = ap_hi
        
    return row

df = df.apply(ap_hi, axis=1)

In [ ]:
df.loc[big_app_hi.index].head()
big_app_hi = None

In [ ]:
df.boxplot('ap_lo')
plt.title('Нижнее значение давления')
plt.show()

In [ ]:
df['ap_lo'].hist(bins=100)
plt.title('Нижнее значение давления')
plt.show()

In [ ]:
# разбираемся с аномально высокими значениями
big_app_lo = df[df['ap_lo'] > 500]
display(big_app_lo.sort_values('ap_lo').head())
print(big_app_lo.shape)

In [ ]:
def ap_lo(row):
    """
    Нормализуем значения в давлении
    
    Параметры:
    ----------
    row: Series
    
    Результат:
    ----------
    Обновлённая строка
    """
    
    if row['ap_lo'] > 500 and row['ap_lo'] <= 2000:
        ap_lo = int(row['ap_lo'] / 10)
        row['ap_lo'] = ap_lo
    elif row['ap_lo'] > 2000:
        ap_lo = int(row['ap_lo'] / 100)
        row['ap_lo'] = ap_lo
        
    return row

df = df.apply(ap_lo, axis=1)

In [ ]:
display(df.loc[big_app_lo.index].head())
big_app_lo = None

In [ ]:
ap_warn = df[df['ap_hi'] < df['ap_lo']]
# теперь разберёмся с ситуацией, когда ap_hi < ap_lo
display(ap_warn.head())
print(ap_warn.shape)

In [ ]:
def ap_mv(row):
    """
    Поменяем местами давление
    
    Параметры:
    ----------
    row: Series
    
    Результат:
    ----------
    Обновлённая строка
    """
    
    if row['ap_hi'] < row['ap_lo'] and row['ap_lo'] > 90:
        ap_lo = row['ap_lo']
        ap_hi = row['ap_hi']
        
        row['ap_hi'] = ap_lo
        row['ap_lo'] = ap_hi
        
    return row

df = df.apply(ap_mv, axis=1)

In [ ]:
display(df.loc[ap_warn.index].head())
ap_warn = None

In [ ]:
# теперь разберёмся с предтрупными значениями
dead_ap = df[(df['ap_hi'] < 60) | (df['ap_lo'] < 40)]
display(dead_ap.sort_values('ap_hi').head())

print(dead_ap.shape)

In [ ]:
# применим самый простой способ
df.loc[dead_ap.index, 'ap_hi'] = None
df.loc[dead_ap.index, 'ap_lo'] = None

df['ap_hi'] = df['ap_hi'].fillna(df.loc[df['is_train'] == 1, 'ap_hi'].median())
df['ap_lo'] = df['ap_lo'].fillna(df.loc[df['is_train'] == 1, 'ap_lo'].median())

display(df.loc[dead_ap.index].head())

dead_ap = None

In [ ]:
df.isna().sum()

Нормализация данных завершена, приступим графическому анализу

### Анализ

In [ ]:
df.corr()

In [ ]:
df['age'] = df['age'].astype('int8')
df['is_male'] = df['is_male'].astype('int8')
df['height'] = df['height'].astype('int16')
df['weight'] = df['weight'].astype('int16')
df['ap_hi'] = df['ap_hi'].astype('int16')
df['ap_lo'] = df['ap_lo'].astype('int16')
df['cholesterol'] = df['cholesterol'].astype('int8')
df['gluc'] = df['gluc'].astype('int8')
df['is_smoke'] = df['is_smoke'].astype('int8')
df['is_alco'] = df['is_alco'].astype('int8')
df['is_cardio'] = df['is_cardio'].astype('float32')
df['is_train'] = df['is_train'].astype('int8')
df['is_active'] = df['is_active'].astype('int8')

In [ ]:
df.describe()

In [ ]:
# для анализа введём новую колонку is_healthy - противоположность is_cardio
df.loc[df['is_cardio'] == 1.0, 'is_healthy'] = 0.0
df.loc[df['is_cardio'] == 0.0, 'is_healthy'] = 1.0
df['is_healthy'] = df['is_healthy'].astype('float32')

In [ ]:
df.info()

In [ ]:
# создадим вспомагательные функции для построения графиков
def graf_hist(df, column, column_title='', figsize=(20,5)):
    """
    Построение гистограммы распределения
    
    Параметры:
    ----------
    df: DataFrame - данные
    column: String - колонка для анализа
    column_title: String - пользовательское имя колонки
    figsize: turtle - размер
    """
    fig = plt.figure(figsize = figsize)
    ax = fig.gca()
    
    if column_title == '':
        column_title = column
    df[column].hist(bins=100, ax=ax)
    
    plt.title(f'Распределение столбца - {column_title}')
    plt.ylabel('Количество')
    plt.xlabel(column_title)
    plt.xticks(rotation=90)
    plt.show()
    
def graf_bar(df, column, column_title=''):
    """
    Построение графика для определения распределения пола в долях
    
    Параметры:
    ----------
    df: DataFrame - данные
    column: String - колонка для анализа
    column_title: String - пользовательское имя колонки
    """
    if column_title == '':
        column_title = column
        
    fig = plt.figure(figsize = (20,5))
    ax = fig.gca()
    df.pivot_table(index=[column], values=['is_healthy', 'is_cardio'], aggfunc='sum') \
        .apply(lambda x: x * 100 / sum(x), axis=1) \
        .plot(kind="bar", ax=ax, stacked=True, grid=True).legend(
            loc='upper center', ncol=3, title="Признак здоровья"
        )

    plt.title(f'Распределение столбца - {column_title}')
    plt.ylabel('Доля')
    plt.xlabel(column_title)
    plt.xticks(rotation=90)
    plt.show()

#### age

In [ ]:
df.isna().sum()

In [ ]:
graf_hist(df, 'age', 'Возрастная группа')

In [ ]:
graf_bar(df, 'age', 'Возрастная группа')

Удалять выбросы с возрастом до 35 лет не будем, т. к. судя по графику (столбики 24 и 25) они присутствуют и в тестовой выборки.

**Вывод**: основную доля людей находится в возрасте от 40 до 65. Чем выше возраст, тем выше риск заболевания ССЗ.

#### Давление

In [ ]:
df.boxplot('ap_hi')
plt.title('Верхнее давление')
plt.show()

graf_hist(df, 'ap_hi', 'Верхнее давление')
graf_bar(df, 'ap_hi', 'Верхнее давление')

**Вывод**: чем давление выше, тем риск ССЗ выше

In [ ]:
df.boxplot('ap_lo')
plt.title('Нижнее давление')
plt.show()


graf_hist(df, 'ap_lo', 'Нижнее давление')
graf_bar(df, 'ap_lo', 'Нижнее давление')

**Вывод**: чем давление выше, тем риск ССЗ выше, и наоборот при  давлении ниже 60 риск повышается.

#### is_male

In [ ]:
graf_hist(df, 'is_male', 'Пол')
graf_bar(df, 'is_male', 'Пол')

**Вывод**: пол со значением 0 больше, чем 1. Риск заболеваний ССЗ одинаковый. При обучении не будем учитывать этот признак

#### height и weight

In [ ]:
graf_hist(df, 'height', 'Рост')
graf_bar(df, 'height', 'Рост')

**Вывод**: Рост незначительно влияет на целевой признак. При этом при аномально низких и высоких показателей роста, риск ССЗ повышается.

In [ ]:
graf_hist(df, 'weight', 'Вес')
graf_bar(df, 'weight', 'Вес')

**Вывод**: если рост отклоняется от среднего, то риск выше. Если вес большой, то риск ССЗ тоже высок.

#### cholesterol

In [ ]:
graf_hist(df, 'cholesterol', 'Холестерин')
graf_bar(df, 'cholesterol', 'Холестерин')

**Вывод**: чем выше холестерин, тем риск ССЗ выше.

#### gluc

In [ ]:
graf_hist(df, 'gluc', 'Глюкоза')
graf_bar(df, 'gluc', 'Глюкоза')

**Вывод**: чем выше глюкоза, тем риск ССЗ выше.

#### smoke

In [ ]:
graf_hist(df, 'is_smoke', 'Курит')
graf_bar(df, 'is_smoke', 'Курит')

**Вывод**: странно, но курение не повышает риск ССЗ

#### is_alco

In [ ]:
graf_hist(df, 'is_alco', 'Злоупотребляет алкоголем')
graf_bar(df, 'is_alco', 'Злоупотребляет алкоголем')

**Вывод**: аналогично с курением

#### is_active

In [ ]:
graf_hist(df, 'is_active', 'Ведёт активных образ жизни')
graf_bar(df, 'is_active', 'Ведёт активных образ жизни')

**Вывод**: занятие спортом незначительно снижает риск ССЗ

#### is_cardio

In [ ]:
graf_hist(df, 'is_cardio', 'Есть ССЗ')

**Вывод**: дисбаланса нет

### ML

Построим три модели и найдём лучшую:

* DecisionTreeClassifier - дерево решений
* RandomForestClassifier - случайный лес
* LogisticRegression - логистическая регрессия

In [ ]:
# создаём вспомогательные методы для хранения лучших метрик
dict_score = {} # переменная для хранения метрик

def best_score(classifier_name, score, method):
    """
    Сохранение лучшей метрики для метода
    
    Параметры:
    ----------
    classifier_name: string - имя классификатора
    score: float - значение метрики
    method: string - метод или группа в рамках, которой производится вычисление 
    """
    item = dict_score.get(method)

    if item:
        if score > item[1]:
            dict_score[method] = [classifier_name, score]
    else:
        dict_score[method] = [classifier_name, score]
        
def get_best_score(method):
    """
    Получение лучшей метрики метода
    
    Параметры:
    ----------
    method: string - метод или группа в рамках, которой производится вычисление 
    
    Результаты
    ----------
    classifier_name: string - имя классификатора
    score: float - значение метрики
    """
    item = dict_score.get(method)
    if item:
        return item[0], item[1]
    else:
        return 'unknown', 0.0

def info_best_score(method):
    """
    Вывод информации о наилучшей метрики
    
    Параметры:
    ----------
    method: string - метод или группа в рамках, которой производится вычисление 
    """
    classifier_name, score = get_best_score(method) 
    print(f'Вывод: наилучшую метрику AUC-ROC показал классификатор {classifier_name} со значением {round(score, 3)}')

In [ ]:
# создаём вспомогательные методы для анализа качества моделей DecisionTreeClassifier
def decisionTreeClassifierScore(features_train, target_train, features_valid, target_valid, max_depth=20, depth_step=1):
    """Поиск оптимального качества модели DecisionTreeClassifier
    
    Параметры:
    ----------
    features_train: DataFrame - обучающая выборка
    target_train: DataFrame - целевой признак обучающей выборки
    features_valid: DataFrame - валидационная выборка
    target_valid: DataFrame - целевой признак валидационной выборки
    max_depth: int, default=20 - глубина обучения
    depth_step: int - шаг
    
    Результаты
    ----------
    best_auc_roc_score: array - метрика AUC-ROC
    chart_data: array - данные для построения графика
    """

    best_auc_roc_score = [0, 0] # AUC-ROC и Глубина
    chart_data = []
    with tqdm(total=int(max_depth / depth_step) - 1) as pbar:
        for depth in range(1, max_depth, depth_step):
            model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
            model.fit(features_train, target_train)

            probabilities_valid = model.predict_proba(features_valid)
            auc_roc = roc_auc_score(target_valid, probabilities_valid[:, 1])

            chart_data.append([depth, auc_roc])

            if auc_roc > best_auc_roc_score[0]:
                best_auc_roc_score = [auc_roc, depth]
                
            pbar.update(1)
            
    return best_auc_roc_score, chart_data

def decisionTreeClassifierChart(data):
    """Построение визуального графика для просмотра качества модели DecisionTreeClassifier
    
    Параметры:
    ----------
    data: array - массив данных из трёх колонок: max_depth, AUC-ROC
    """
    
    df = pd.DataFrame(data, columns=['max_depth', 'auc_roc'])
    plt.title('Распределение метрик качества')
    plt.xlabel('Глубина обучения')
    plt.ylabel('Значение метрики')
    plt.plot(df['max_depth'], df['auc_roc'], label = "AUC-ROC")
    plt.legend()
    plt.show()

def decisionTreeClassifierInfo(auc_roc_score):
    """Вывод
    
    Параметры:
    ----------
    auc_roc_score: array - показатели для AUC-ROC 
    """
    print(f'''Для классификатора DecisionTreeClassifier:
 - лучшее качество AUC-ROC равно {round(auc_roc_score[0], 3)}, которое было найдено при гиперпараметре max_depth равном {auc_roc_score[1]}.''')

In [ ]:
# создаём вспомогательные методы для анализа качества моделей RandomForestClassifier

def randomForestClassifierScore(features_train, target_train, features_valid, target_valid, max_depth=10, n_estimators=30, step=2):
    """Поиск оптимального качества модели RandomForestClassifier
    
    Параметры:
    ----------
    features_train: DataFrame - обучающая выборка
    target_train: DataFrame - целевой признак обучающей выборки
    features_valid: DataFrame - валидационная выборка
    target_valid: DataFrame - целевой признак валидационной выборки
    max_depth: int, default=10 - глубина обучения
    n_estimators: int, default=30 - количество лесов
    step: int - шаг
    
    Результаты
    ----------
    best_auc_roc_score: array - метрика AUC-ROC
    auc_roc_chart_data: array - данные для построения графика по метрике AUC-ROC
    """
    
    best_auc_roc_score = [0, 0, 0] # AUC-ROC, Глубина и леса
    auc_roc_chart_data = []
    
    estimators_len = int(n_estimators / step) - 1
    if estimators_len <= 0:
        estimators_len = 1
    
    with tqdm(total=estimators_len * (int(max_depth / step) - 1)) as pbar:
        for est in reversed(range(1, n_estimators, step)):
            auc_roc_data = []

            for depth in range(1, max_depth, step):
                model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
                model.fit(features_train, target_train)

                probabilities_valid = model.predict_proba(features_valid)
                auc_roc = roc_auc_score(target_valid, probabilities_valid[:, 1])

                if auc_roc > best_auc_roc_score[0]:
                    best_auc_roc_score = [auc_roc, depth, est]

                auc_roc_data.append(auc_roc)
                
                pbar.update(1)

            auc_roc_chart_data.append(auc_roc_data)

    return best_auc_roc_score, auc_roc_chart_data

def randomForestClassifierChart(auc_roc_data, max_depth=10, n_estimators=30, step=2):
    """Построение визуального графика для просмотра качества модели RandomForestClassifier
    
    Параметры:
    ----------
    auc_roc_data: array - массив данных для метрики AUC-ROC
    """
    
    fig, axs = plt.subplots(figsize=(20, 6)) 

    df = pd.DataFrame(auc_roc_data, index=list(reversed(range(1, n_estimators, step))), columns=list(range(1, max_depth, step)))
    sbn.heatmap(df, ax=axs)
    axs.set_title('Распределение метрики AUC-ROC')
    axs.set_xlabel('Глубина обучения')
    axs.set_ylabel('Количество деревьев')

    plt.show()

def randomForestClassifierInfo(auc_roc_score):
    """Вывод для модели RandomForestClassifier
    
    Параметры:
    ----------
    auc_roc_score: array - показатели для AUC-ROC 
    """
    print(f'''Для классификатора RandomForestClassifier:
 - лучшее качество AUC-ROC равно {round(auc_roc_score[0], 3)}, которое было найдено при гиперпараметрах n_estimators равном {round(auc_roc_score[2], 3)} и max_depth равном {auc_roc_score[1]}.''')

In [ ]:
# создаём вспомогательные методы для анализа качества моделей LogisticRegression

def logisticRegressionScore(features_train, target_train, features_valid, target_valid, max_iter=1000):
    """Поиск оптимального качества модели LogisticRegression
    
    Параметры:
    ----------
    features_train: DataFrame - обучающая выборка
    target_train: DataFrame - целевой признак обучающей выборки
    features_valid: DataFrame - валидационная выборка
    target_valid: DataFrame - целевой признак валидационной выборки
    
    Результаты
    ----------
    best_auc_roc_score: int - метрика AUC-ROC
    """
    
    best_auc_roc_score = 0 # AUC-ROC
    
    model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=max_iter)
    model.fit(features_train, target_train)

    probabilities_valid = model.predict_proba(features_valid)
    best_auc_roc_score = roc_auc_score(target_valid, probabilities_valid[:, 1])

    return best_auc_roc_score

def logisticRegressionInfo(auc_roc_score):
    """Вывод для модели LogisticRegression
    
    Параметры:
    ----------
    auc_roc_score: array - показатели для AUC-ROC 
    """
    print(f'''Для классификатора LogisticRegression:
 - лучшее качество AUC-ROC равно {round(auc_roc_score, 3)}.''')

In [ ]:
df.info()

In [ ]:
# разделим выборки
df_train = df.loc[df['is_train'] == 1]
df_test = df.loc[df['is_train'] == 0]

In [ ]:
df_train.info()

In [ ]:
df_train.head()

In [ ]:
df_train.corr()

In [ ]:
drop_columns = ['is_train', 'is_healthy', 'is_cardio']

In [ ]:
features = df_train.drop(drop_columns, axis=1)
target = df_train['is_cardio']   

In [ ]:
features.info()

In [ ]:
# делим для валидации
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.4, random_state=12345)
# делим для тестирования 
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5, random_state=12345)

In [ ]:
print('\nDecisionTreeClassifier')

# используем созданный ранее метод
dtc_best_auc_roc_score, chart_data = decisionTreeClassifierScore(features_train, target_train, features_valid, target_valid, max_depth=20, depth_step=1)
# строим график
decisionTreeClassifierChart(chart_data)


# сохраняем лучший результат
best_score('DecisionTreeClassifier', dtc_best_auc_roc_score[0], 'Базовый')
# вывод информации
decisionTreeClassifierInfo(dtc_best_auc_roc_score)

print('\nRandomForestClassifier')

# используем созданный ранее метод
rfc_best_auc_roc_score, auc_roc_chart_data = randomForestClassifierScore(features_train, target_train, features_valid, target_valid, max_depth=10, n_estimators=19, step=1)
# строим график
randomForestClassifierChart(auc_roc_chart_data, max_depth=10, n_estimators=19, step=1)

# сохраняем лучший результат
best_score('RandomForestClassifier', rfc_best_auc_roc_score[0], 'Базовый')
# вывод информации
randomForestClassifierInfo(rfc_best_auc_roc_score)

print('\nLogisticRegression')

scaler = StandardScaler()
features_train_ss = features_train.copy()
features_valid_ss = features_valid.copy()

scale_columns = ['age', 'weight', 'height', 'ap_hi', 'ap_lo', 'gluc', 'cholesterol']

scaler.fit(features_train_ss.loc[:, scale_columns])
features_train_ss[scale_columns] = scaler.transform(features_train_ss[scale_columns])
features_valid_ss[scale_columns] = scaler.transform(features_valid_ss[scale_columns])

lr_best_auc_roc_score = logisticRegressionScore(features_train_ss, target_train, features_valid_ss, target_valid)

# сохраняем лучший результат
best_score('LogisticRegression', lr_best_auc_roc_score, 'Базовый')
# вывод информации
logisticRegressionInfo(lr_best_auc_roc_score)

print('\n')
info_best_score('Базовый') 

#### Тестирование модели

In [ ]:
# получаем лучшую метрику
best_method = 'unknown'
best_classifier = 'unknown'
best_score_method = 0

for method in dict_score:
    item = get_best_score(method)
    if item[1] > best_score_method:
        best_score_method = item[1]
        best_classifier = item[0]
        best_method = method
        
print(f'Лучшую метрику ROC-AUC {round(best_score_method, 3)} показал метод "{best_method}" с классификатором {best_classifier}.')

In [ ]:
MAX_DEPTH=8
N_ESTIMATORS=16

In [ ]:
model = RandomForestClassifier(random_state=12345, max_depth=MAX_DEPTH, n_estimators=N_ESTIMATORS)
model.fit(features_train, target_train)

probabilities_valid = model.predict_proba(features_test)
auc_roc = roc_auc_score(target_test, probabilities_valid[:, 1])

print(f'AUC-ROC =', auc_roc)

print(classification_report(target_test, model.predict(features_test)))

In [ ]:
# обучим на всей выборке
features = df_train.drop(drop_columns, axis=1)
target = df_train['is_cardio']

cls = RandomForestClassifier(random_state=12345, max_depth=MAX_DEPTH, n_estimators=N_ESTIMATORS)
cls.fit(features, target)

In [ ]:
features = df_test.drop(drop_columns, axis=1)

predictions = cls.predict(features)

to_submit = pd.DataFrame(index=df_test.index)
to_submit['cardio'] = predictions
to_submit['cardio'] = to_submit['cardio'].astype('float16')
to_submit.to_csv('submission.csv')
print(to_submit.shape)

dump(cls, 'heart.model')

In [ ]:
to_submit.head()